In [2]:
import numpy as np   
import pandas as pd        
from scipy import stats as sts 

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline 

from math import factorial as fact

## Задачи на проверку гипотез

### Задача №1: Джеймс Бонд

<img src="../images/bond.png" width="300" align='left'>

Ваш друг Джеймс Бонд утверждает, что умеет отличать взболтанный мартини от смешанного. Вы решили проверить это. Чтобы сделать это, вы завязали глаза Бонду и дали попробовать ему мартини 10 раз. Пусть Бонд отгадал тип мартини 6 раз и не угадал 4. Давайте проверим гипотезу о том, что Бонд умеет отличать мартини, используя z-test.

*Выборка:* $1, 0, 0, 1, 1, 0, 0, 1, 1, 1$

In [3]:
X = np.array([1, 0, 0, 1, 1, 0, 0, 1, 1, 1])
n = len(X)
alpha = 0.05

**Решение:**

$$
H_0: p \le 0.5 \\
H_1: p > 0.5
$$

$$
H_0: p = 0.5 \\ 
H_1: p > 0.5 
$$


$$
z = \frac{\hat p - 0.5}{\sqrt{\frac{0.5 (1 - 0.5)}{10}}} \sim_{n \rightarrow \infty} \mathcal{N}(0, 1)
$$

In [4]:
p_hat = np.mean(X)
p_0 = 0.5
denom = ((p_0 * (1-p_0)) / 10)**(1/2)

z_obs = (p_hat - p_0) / denom
z_cr = sts.norm.ppf(q=(1-alpha), loc=0, scale=1)

if z_obs > z_cr:
    print(f'{round(z_obs, 2)} > {round(z_cr, 2)} => отвергаем нулевую гипотезу')
else:
    print(f'{round(z_obs, 2)} < {round(z_cr, 2)} => не отвергаем нулевую гипотезу')

0.63 < 1.64 => не отвергаем нулевую гипотезу


То же самое, но одной функцией:

In [5]:
from statsmodels.stats.proportion import proportions_ztest

proportions_ztest(sum(X), len(X), value=0.5, alternative='larger')

(0.6454972243679027, 0.25930250821436285)

### Задача №2: О мышах и людях

<img src="../images/vagon.jpeg" width="500" align='left'>

Для изучения аспектов процесса принятия моральных решений психологи уже много лет используют этические дилеммы, с помощью которых оценивают действия людей в гипотетических ситуациях. Один из самых известных примеров — это [проблема вагонетки,](https://ru.wikipedia.org/wiki/Проблема_вагонетки) в которой необходимо принять решение о том, стоит ли пожертвовать одним человеком для спасения пятерых.

Бельгийские психологи воплотили дилемму в реальную жизнь. Участники эксперимента должны были выбрать, ударить током пять мышей или одну мышь. Эксперимент проходил следующим образом. Участника сажали перед двумя клетками, в одной из которых сидели пять мышей, а в другой — одна. Перед клетками стоял ноутбук с $20$-секундным таймером: участникам сообщили, что по истечении этих $20$ секунд в клетку с пятью мышами пустят ток, и мыши получат не смертельный, но очень болезненный удар. Пока время идет, участник может нажать на кнопку: в этом случае ток пустят по клетке с одной мышью. В исследовании использовали живых мышей. 

Удары тока были ненастоящими: сразу же после «удара» участников сопроводили в отдельную комнату, где разъяснили им, что мыши в полном порядке и током их не били (об этом заранее догадались только $12$ участников). В решении реальной проблемы вагонетки приняли участие $192$ человека, а еще $83$ решали такую же задачку, но гипотетическую (на бумаге). Все участники также прошли онлайн-опросы, в ходе которых учёные собрали о респондентах кучу дополнительной информации. 

В файле `mouse.csv` лежит информация о том, как прошёл эксперимент. Нас будут интересовать столбцы: 

* __STUDY:__ какую проблему вагонетки решал человек $1$, если на бумаге и $2$, если реальную
* __AGE:__ возраст респондента 
* __GENDER:__ пол респондента
* __DECISION:__ решение дилеммы ($1$ - жать на кнопу, $0$ - не жать) 
* __RT:__ время, которое респондент потратил, чтобы нажать на кнопку 

Подробное описание данных, сами данные и даже код на R, использованный при оценивании моделей, можно найти в [репозитории исследования.](https://osf.io/kvb99/) В статье авторы строили несколько логистических регрессий, чтобы очистить эффект от психологических особенностей респондентов. Про подобные приёмы очистки мы немного поговорим позже.  Также более подробно про исследование [можно почитать на N + 1.](https://nplus1.ru/news/2018/05/11/mice-trolley)

In [38]:
df = pd.read_csv('../data/mouse.csv', sep='\t')

# отбираем нужные колонки
df = df[['STUDY', 'AGE', 'GENDER', 'DECISION', 'RT']]

# удаляем пропуски по колонке DECISION
#     (то есть тех, кто не смог принять решение)
df = df[~df.DECISION.isnull()]

### Гипотеза №1: Доля и нажатия на кнопку

Попытаемся ответить на тот же вопрос, что и исследователи. В какой из двух ситуаций респонденты чаще принимают решение нажать кнопку: в реальной или на бумаге?

$$
\begin{aligned}
&H_0: \hspace{2mm} p_1 = p_2 \hspace{2mm} \text{(На бумаге и в реальности жмут на кнопку одинаково часто)} \\
&H_1: \hspace{2mm} p_1 > p_2  \hspace{2mm} \text{(На бумаге на кнопку жмут чаще)}
\end{aligned}
$$

In [39]:
sample_paper = df[df.STUDY == 1].DECISION.values
sample_real = df[df.STUDY == 2].DECISION.values

In [40]:
print(f'Доля нажатий на бумаге: {np.mean(sample_paper)}')
print(f'Доля нажатий в реальности: {np.mean(sample_real)}')

Доля нажатий на бумаге: 0.8645833333333334
Доля нажатий в реальности: 0.7469879518072289


In [41]:
proportions_ztest(
    (sum(sample_paper), sum(sample_real)), 
    (len(sample_paper), len(sample_real)), 
    value=0, 
    alternative='larger'
)

(2.3780989461645565, 0.008701077805778048)

> __Вывод:__ гипотеза о том, что на кнопку жмут одинаково часто, отвергается. На бумаге на кнопку жмут чаще.

### Гипотеза №2. Среднее и кровожадность

Кровожадные люди быстро берут на себя ответственность за удар мышки током. Будем считать, что кровожадные люди принимают решение менее, чем за пять секунд. Правда ли, что люди по своей природе кровожадные? 

$$
\begin{aligned}
&H_0: \hspace{2mm} \mu \le 5 \hspace{2mm} \text{(Люди кровожадны)} \\
&H_1: \hspace{2mm} \mu > 5  \hspace{2mm} \text{(Люди не кровожадны)}
\end{aligned}
$$

In [54]:
X = df[~df.RT.isnull()].RT.values
n = len(X)

rt_mean = np.mean(X)
print(f'Среднее кол-во секунд по выборке: {rt_mean}')

Среднее кол-во секунд по выборке: 10.116762704819276


In [55]:
t_obs = (rt_mean - 5) / (np.std(X, ddof=1) / n**(1/2))
t_cr = sts.t.ppf(q=(1 - alpha), df=n-1)

if t_obs > t_cr:
    print(f'{round(t_obs, 2)} > {round(t_cr, 2)} => отвергаем, люди не кровожадны')
else:
    print(f'{round(t_obs, 2)} < {round(t_cr, 2)} => не отвергаем, люди кровожадны')

12.97 > 1.65 => отвергаем, люди не кровожадны


In [56]:
sts.ttest_1samp(X, 5, axis=0, alternative='greater')

Ttest_1sampResult(statistic=12.974895950977869, pvalue=2.7586881778173365e-27)